In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Étape 1 : Chargement des données

In [2]:
def load_data(file_path, sep=",", encoding="ISO-8859-1", header=None, rename_columns=None):
    """Charge les données depuis un fichier CSV."""
    try:
        data = pd.read_csv(file_path, sep=sep, encoding=encoding, header=header)
        # Apply column renaming during loading
        if rename_columns:
            data.columns = rename_columns
            print(f"Colonnes renommées : {rename_columns}")
        print(f"Données chargées avec succès depuis '{file_path}'.")
        return data
    except Exception as e:
        print(f"Erreur lors du chargement des données : {e}")
        return None

# Étape 2 : Sélection des colonnes à conserver 


In [3]:
def keep_only_necessary_columns(data, columns_to_keep):
    """
    Garde uniquement les colonnes spécifiées dans le DataFrame.
    """
    try:
        initial_columns = data.columns.tolist()
        data = data[columns_to_keep]
        print(f"\nColonnes conservées : {columns_to_keep}")
        dropped_columns = [col for col in initial_columns if col not in columns_to_keep]
        if dropped_columns:
            print(f"Colonnes supprimées : {dropped_columns}")
        return data
    except Exception as e:
        print(f"Erreur lors de la sélection des colonnes : {e}")
        return data

# Étape 3 : Nettoyage des données 

In [4]:
def clean_data(data, rules):
    """
    Effectue le nettoyage des données selon les règles spécifiques.
    """
    # Ensure required columns exist
    required_columns = rules.get("columns_to_keep", [])
    missing_columns = [col for col in required_columns if col not in data.columns]
    if missing_columns:
        print(f"Erreur : Les colonnes suivantes sont manquantes dans les données : {missing_columns}")
        return data  # Return the data without cleaning

    # Renommage des colonnes si nécessaire
    if "rename_columns" in rules:
        data.columns = rules["rename_columns"]
        print("Colonnes renommées.")

    # Suppression des doublons
    data = data.drop_duplicates()
    print(f"Doublons supprimés. Nouvelle taille : {data.shape}")

    # Remplacement des valeurs manquantes
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    categorical_cols = data.select_dtypes(include=["object"]).columns

    for col in numeric_cols:
        if col in rules.get("numeric_fill", {}):
            data[col].fillna(rules["numeric_fill"][col], inplace=True)
        else:
            data[col].fillna(0, inplace=True)  # Valeur par défaut

    for col in categorical_cols:
        data[col] = data[col].str.strip()  # Strip spaces
        data[col].replace("", "-", inplace=True)  # Replace empty strings with '-'
        data[col].fillna("Non spécifié", inplace=True)

    # Custom cleaning logic for Depot
    if "custom_cleaning_steps" in rules:
        for step in rules["custom_cleaning_steps"]:
            if step == "handle_currency_and_country":
                def update_devise(row):
                    if pd.isna(row['GDE_PAYS']) or str(row['GDE_PAYS']).strip() in ['', '-']:
                        return row['GDE_DEVISE']
                    elif row['GDE_PAYS'] == 'CIV': return 'XOF'
                    elif row['GDE_PAYS'] == 'TUN': return 'TND'
                    return row['GDE_DEVISE']
                
                data['GDE_DEVISE'] = data.apply(update_devise, axis=1)
                
                def fill_country(row):
                    if row['GDE_PAYS'] in ['-', 'nan'] and row['GDE_DEVISE'] == 'TND':
                        return 'TUN'
                    return row['GDE_PAYS']
                
                data['GDE_PAYS'] = data.apply(fill_country, axis=1)
                print("Custom currency/country logic applied")
            
            elif step == "clean_missing_values":
                data['GDE_PAYS'] = data['GDE_PAYS'].str.strip()
                data.replace(['', 'nan', 'NaN', None], '-', inplace=True)
                data.fillna('-', inplace=True)
                print("Missing values cleaned")

    # Identification et suppression des anomalies
    if "anomalies" in rules:
        try:
            anomalies = data.query(rules["anomalies"])
            if not anomalies.empty:
                print(f"{len(anomalies)} anomalies détectées.")
                data = data[~data.index.isin(anomalies.index)]
        except Exception as e:
            print(f"Erreur lors de l'application des anomalies : {e}")

    print("Nettoyage des données terminé.")
    return data

# Étape 4 : Transformation des données 

In [5]:
def transform_data(data, rules):
    """
    Transforme les données : encodage, création de nouvelles features, etc.
    """
    # Encodage des variables catégoriques
    if "categorical_columns" in rules:
        for col in rules["categorical_columns"]:
            le = LabelEncoder()
            data[col] = le.fit_transform(data[col].astype(str))
        print("Encodage des variables catégoriques terminé.")

    # Conversion des dates en format datetime
    if "date_columns" in rules:
        for col in rules["date_columns"]:
            data[col] = pd.to_datetime(data[col], errors="coerce")

    # Création de nouvelles features
    if "new_features" in rules:
        for feature_name, feature_formula in rules["new_features"].items():
            data[feature_name] = eval(feature_formula)
        print("Création de nouvelles features terminée.")

    print("Transformation des données terminée.")
    return data

# Étape 5 : Validation des données 

In [6]:
def validate_data(data):
    """
    Valide les données pour s'assurer qu'elles sont prêtes pour l'analyse.
    """
    # Vérification des valeurs manquantes
    missing_values = data.isnull().sum().sum()
    if missing_values > 0:
        print(f"Attention : {missing_values} valeurs manquantes restantes.")
    else:
        print("Aucune valeur manquante dans les données.")

    # Vérification des types de données
    print("Types de données dans le DataFrame :")
    print(data.dtypes)

    print("Validation des données terminée.")

# Pipeline principal 

In [7]:
def preprocess_data(file_path, output_file, rules):
    """
    Prétraite les données selon les règles spécifiques.
    """
    # Étape 1 : Chargement des données
    data = load_data(file_path, **rules.get("load_params", {}))
    if data is None:
        return None

    # Étape 2 : Sélection des colonnes à conserver
    if "columns_to_keep" in rules:
        data = keep_only_necessary_columns(data, rules["columns_to_keep"])

    # Étape 3 : Nettoyage des données
    data = clean_data(data, rules)

    # Étape 4 : Transformation des données
    data = transform_data(data, rules)

    # Étape 5 : Validation des données
    validate_data(data)

    # Sauvegarde des données prétraitées
    data.to_csv(output_file, index=False)
    print(f"Données prétraitées sauvegardées dans '{output_file}'.")

    return data

# Configuration des règles pour chaque fichier 

In [8]:
RULES = {
    "Article": {
        "load_params": {"header": 0},
        "columns_to_keep": ["ID_Article", "Nom_Article", "Prix_vente_TTC", "Prix_achat_HT", "Categorie", "Quantite_Stock", "Date_Creation"],
        "numeric_fill": {"Prix_vente_TTC": 0, "Prix_achat_HT": 0},
        "anomalies": "(Prix_vente_TTC < 0) | (Prix_achat_HT < 0)",
        "categorical_columns": ["Categorie"],
        "date_columns": ["Date_Creation"],
        "new_features": {
            "Marge_Brute": "data['Prix_vente_TTC'] - data['Prix_achat_HT']",
            "Taux_Marge": "data['Marge_Brute'] / data['Prix_achat_HT'].replace(0, np.nan)"
        }
    },
    "Dispos": {
        "load_params": {"header": 0},
        "columns_to_keep": ["GQ_STOCKMIN", "GQ_STOCKMAX", "GQ_PHYSIQUE", "GQ_DATECREATION"],
        "numeric_fill": {"GQ_STOCKMIN": 0, "GQ_STOCKMAX": "data['GQ_PHYSIQUE'] * 2"},
        "anomalies": "(GQ_PHYSIQUE > GQ_STOCKMAX) | (GQ_PHYSIQUE < GQ_STOCKMIN)",
        "date_columns": ["GQ_DATECREATION"],
        "new_features": {
            "Taux_Remplissage": "data['GQ_PHYSIQUE'] / data['GQ_STOCKMAX'].replace(0, np.nan)",
            "Stock_Disponible": "data['GQ_PHYSIQUE'] - data['GQ_STOCKMIN']"
        }
    },
    "Tiers": {
        "load_params": {"header": None},
        "columns_to_keep": ["T_AUXILIAIRE", "T_TIERS", "T_DEVISE", "T_NATURE", "T_REMISE", "T_VILLE", "T_PAYS", "T_DATECREATION", "T_DATEFERMETURE"],
        "numeric_fill": {},
        "anomalies": "(T_DATECREATION > T_DATEFERMETURE) & (T_DATEFERMETURE != '1900-01-01 00:00:00')",
        "categorical_columns": ["T_DEVISE", "T_NATURE", "T_VILLE", "T_PAYS"],
        "date_columns": ["T_DATECREATION", "T_DATEFERMETURE"],
        "new_features": {
            "Anciennete": "(pd.Timestamp.now() - data['T_DATECREATION']).dt.days / 365",
            "Statut_Actif": "np.where(data['T_DATEFERMETURE'] == '1900-01-01 00:00:00', 1, 0)"
        }
    },
    "Depot": {
        "load_params": {
            "header": None,
            "rename_columns": ["GDE_DEPOT", "GDE_VILLE", "GDE_PAYS", "GDE_DEVISE"]
        },
        "columns_to_keep": ["GDE_DEPOT", "GDE_VILLE", "GDE_PAYS", "GDE_DEVISE"],
        "numeric_fill": {},
        "anomalies": "(GDE_PAYS == '-') | (GDE_DEVISE == '-')",
        "categorical_columns": ["GDE_PAYS", "GDE_DEVISE"],
        "custom_cleaning": True
    },
    "Ligne": {
        "load_params": {
        "header": None,
        "rename_columns": [
            "GL_NUMERO", "GL_NUMLIGNE", "GL_ARTICLE", "GL_TIERS", "GL_DEPOT",
            "GL_PUHT", "GL_PUTTC", "GL_PRHT", "GL_QTEFACT", "GL_REMISELIGNE"
        ]
    },
    "columns_to_keep": [
        "GL_NUMERO", "GL_NUMLIGNE", "GL_ARTICLE", "GL_TIERS", "GL_DEPOT",
        "GL_PUHT", "GL_PUTTC", "GL_PRHT", "GL_QTEFACT", "GL_REMISELIGNE"
    ],
    "numeric_fill": {
        "GL_PUHT": 0, "GL_PUTTC": 0, "GL_PRHT": 0, "GL_QTEFACT": 0, "GL_REMISELIGNE": 0
    },
    "anomalies": "(GL_PUHT > GL_PUTTC) | (GL_REMISELIGNE < 0) | (GL_REMISELIGNE > 100) | (GL_QTEFACT < 0)",
    "categorical_columns": [],
    "new_features": {}
},
    "Piece": {
        "load_params": {
            "header": None,
            "rename_columns": [
                "GP_NATUREPIECEG", "GP_SOUCHE", "GP_NUMERO", "GP_TOTALHT", 
                "GP_TOTALTTC", "GP_DATEPIECE", "GP_DEPOT", "GP_TIERS", "GP_DEVISE"
            ]
        },
        "columns_to_keep": ["GP_NATUREPIECEG", "GP_SOUCHE", "GP_NUMERO", "GP_TOTALHT", "GP_TOTALTTC", "GP_DATEPIECE", "GP_DEPOT", "GP_TIERS", "GP_DEVISE"],
        "numeric_fill": {"GP_TOTALHT": 0, "GP_TOTALTTC": 0},
        "anomalies": "(GP_TOTALHT > GP_TOTALTTC) | (GP_TOTALHT < 0) | (GP_TOTALTTC < 0) | (GP_DATEPIECE.dt.year < 2023)",
        "date_columns": ["GP_DATEPIECE"],
        "categorical_columns": ["GP_DEVISE"],
        "new_features": {}
    },
    "Remise": {
        "load_params": {
            "header": None,
            "rename_columns": [
                "MLR_ETABLISSEMENT", "MLR_SOUCHE", "MLR_NUMERO", "MLR_NUMLIGNE",
                "MLR_ORGREMISE", "MLR_TYPEREMISE", "MLR_REMISE", "MLR_VALEURREMDEV",
                "MLR_MONTANTHTDEV", "MLR_MONTANTTTCDEV"
            ]
        },
        "columns_to_keep": ["MLR_ETABLISSEMENT","MLR_SOUCHE","MLR_NUMERO","MLR_NUMLIGNE","MLR_ORGREMISE","MLR_TYPEREMISE","MLR_REMISE","MLR_VALEURREMDEV","MLR_MONTANTHTDEV","MLR_MONTANTTTCDEV"],
        "numeric_fill": {"MLR_REMISE":0,"MLR_VALEURREMDEV":0,"MLR_MONTANTHTDEV":0,"MLR_MONTANTTTCDEV":0},
        "anomalies": "(MLR_MONTANTHTDEV > MLR_MONTANTTTCDEV) | (MLR_REMISE <= 0) | (MLR_MONTANTTTCDEV < 0)",
        "categorical_columns": [],
        "new_features": {}
    }
}

#  Exécution principale 

In [9]:
if __name__ == "__main__":
    # Liste des fichiers à prétraiter
    files_to_process = [
        {"file_path": "Article.csv", "output_file": "Article_Preprocessed.csv", "rules": RULES["Article"]},
        {"file_path": "DISPOS.csv", "output_file": "DISPO_Preprocessed.csv", "rules": RULES["Dispos"]},
        {"file_path": "Tier.csv", "output_file": "TIERS_Preprocessed.csv", "rules": RULES["Tiers"]},
        {"file_path": "Depot.csv", "output_file": "Depot_Preprocessed.csv", "rules": RULES["Depot"]},
        {"file_path": "lastLigne.csv", "output_file": "Ligne_Preprocessed.csv", "rules": RULES["Ligne"]},
        {"file_path": "Piece.csv", "output_file": "Piece_Preprocessed.csv", "rules": RULES["Piece"]},
        {"file_path": "Remise.csv", "output_file": "Remise_Preprocessed.csv", "rules": RULES["Remise"]}
    ]

    # Prétraitement de chaque fichier
    for file_info in files_to_process:
        print(f"\n=== Traitement du fichier : {file_info['file_path']} ===")
        preprocess_data(
            file_path=file_info["file_path"],
            output_file=file_info["output_file"],
            rules=file_info["rules"]
        )


=== Traitement du fichier : Article.csv ===
Erreur lors du chargement des données : [Errno 2] No such file or directory: 'Article.csv'

=== Traitement du fichier : DISPOS.csv ===
Erreur lors du chargement des données : [Errno 2] No such file or directory: 'DISPOS.csv'

=== Traitement du fichier : Tier.csv ===
Erreur lors du chargement des données : [Errno 2] No such file or directory: 'Tier.csv'

=== Traitement du fichier : Depot.csv ===
Colonnes renommées : ['GDE_DEPOT', 'GDE_VILLE', 'GDE_PAYS', 'GDE_DEVISE']
Données chargées avec succès depuis 'Depot.csv'.

Colonnes conservées : ['GDE_DEPOT', 'GDE_VILLE', 'GDE_PAYS', 'GDE_DEVISE']
Doublons supprimés. Nouvelle taille : (67, 4)
7 anomalies détectées.
Nettoyage des données terminé.
Encodage des variables catégoriques terminé.
Transformation des données terminée.
Aucune valeur manquante dans les données.
Types de données dans le DataFrame :
GDE_DEPOT     object
GDE_VILLE     object
GDE_PAYS       int32
GDE_DEVISE     int32
dtype: object

C:\Users\azizm\AppData\Local\Temp\ipykernel_19452\1173233911.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].replace("", "-", inplace=True)  # Replace empty strings with '-'
C:\Users\azizm\AppData\Local\Temp\ipykernel_19452\1173233911.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

Colonnes renommées : ['GL_NUMERO', 'GL_NUMLIGNE', 'GL_ARTICLE', 'GL_TIERS', 'GL_DEPOT', 'GL_PUHT', 'GL_PUTTC', 'GL_PRHT', 'GL_QTEFACT', 'GL_REMISELIGNE']
Données chargées avec succès depuis 'lastLigne.csv'.

Colonnes conservées : ['GL_NUMERO', 'GL_NUMLIGNE', 'GL_ARTICLE', 'GL_TIERS', 'GL_DEPOT', 'GL_PUHT', 'GL_PUTTC', 'GL_PRHT', 'GL_QTEFACT', 'GL_REMISELIGNE']
Doublons supprimés. Nouvelle taille : (10236953, 10)


C:\Users\azizm\AppData\Local\Temp\ipykernel_19452\1173233911.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].str.strip()  # Strip spaces
C:\Users\azizm\AppData\Local\Temp\ipykernel_19452\1173233911.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].

Erreur lors de l'application des anomalies : '<' not supported between instances of 'str' and 'int'
Nettoyage des données terminé.
Encodage des variables catégoriques terminé.
Création de nouvelles features terminée.
Transformation des données terminée.
Aucune valeur manquante dans les données.
Types de données dans le DataFrame :
GL_NUMERO         object
GL_NUMLIGNE       object
GL_ARTICLE        object
GL_TIERS          object
GL_DEPOT          object
GL_PUHT           object
GL_PUTTC          object
GL_PRHT           object
GL_QTEFACT        object
GL_REMISELIGNE    object
dtype: object
Validation des données terminée.
Données prétraitées sauvegardées dans 'Ligne_Preprocessed.csv'.

=== Traitement du fichier : Piece.csv ===


...

Erreur lors du chargement des données : Length mismatch: Expected axis has 34 elements, new values have 9 elements

=== Traitement du fichier : Remise.csv ===


C:\Users\azizm\AppData\Local\Temp\ipykernel_19452\2951659249.py:4: DtypeWarning: Columns (0,1,2,3,4,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep=sep, encoding=encoding, header=header)


Colonnes renommées : ['MLR_ETABLISSEMENT', 'MLR_SOUCHE', 'MLR_NUMERO', 'MLR_NUMLIGNE', 'MLR_ORGREMISE', 'MLR_TYPEREMISE', 'MLR_REMISE', 'MLR_VALEURREMDEV', 'MLR_MONTANTHTDEV', 'MLR_MONTANTTTCDEV']
Données chargées avec succès depuis 'Remise.csv'.

Colonnes conservées : ['MLR_ETABLISSEMENT', 'MLR_SOUCHE', 'MLR_NUMERO', 'MLR_NUMLIGNE', 'MLR_ORGREMISE', 'MLR_TYPEREMISE', 'MLR_REMISE', 'MLR_VALEURREMDEV', 'MLR_MONTANTHTDEV', 'MLR_MONTANTTTCDEV']
Doublons supprimés. Nouvelle taille : (3016669, 10)


C:\Users\azizm\AppData\Local\Temp\ipykernel_19452\1173233911.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].replace("", "-", inplace=True)  # Replace empty strings with '-'
C:\Users\azizm\AppData\Local\Temp\ipykernel_19452\1173233911.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

Erreur lors de l'application des anomalies : '<=' not supported between instances of 'str' and 'int'
Nettoyage des données terminé.
Encodage des variables catégoriques terminé.
Création de nouvelles features terminée.
Transformation des données terminée.
Aucune valeur manquante dans les données.
Types de données dans le DataFrame :
MLR_ETABLISSEMENT    object
MLR_SOUCHE           object
MLR_NUMERO           object
MLR_NUMLIGNE         object
MLR_ORGREMISE        object
MLR_TYPEREMISE       object
MLR_REMISE           object
MLR_VALEURREMDEV     object
MLR_MONTANTHTDEV     object
MLR_MONTANTTTCDEV    object
dtype: object
Validation des données terminée.
Données prétraitées sauvegardées dans 'Remise_Preprocessed.csv'.
